<a href="https://colab.research.google.com/github/Vikas1177/Grammar_Auto-Correct/blob/main/Grammar_Auto_Correct_with_T5_large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.31.0

In [ ]:
import spacy
import string
from transformers import T5TokenizerFast,AutoModelForSeq2SeqLM,pipeline
nlp  = spacy.load("en_core_web_sm")

In [ ]:
BATCH_SIZE=64
MAX_LENGTH=128

In [ ]:
model_id="t5-large"
tokenizer=T5TokenizerFast.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set 

In [ ]:
pretrained_model=AutoModelForSeq2SeqLM.from_pretrained(
    "pszemraj/flan-t5-large-grammar-synthesis"
)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


In [ ]:
wrong_english=[]
doc = input("Enter The Text")
doc_nlp = nlp(doc)
for sentences in doc_nlp.sents:
  wrong_english.append(str(sentences))
tokenized=tokenizer(
  wrong_english,
  padding="longest",
  max_length=MAX_LENGTH,
  truncation=True,
  return_tensors='pt'
)
out=pretrained_model.generate(**tokenized, max_length=128)
right_english=[]
for i in range(len(wrong_english)):
  right_english.append(tokenizer.decode(out[i], skip_special_tokens=True))
doc2=''.join(right_english)
print(''.join(right_english))
print(summarizer(str(doc2), max_length=130, min_length=30, do_sample=False))


KeyboardInterrupt: Interrupted by user

In [ ]:
!pip install streamlit
!pip install pyngrok

In [ ]:

import streamlit as st

# Define the function
def grammarly(input_string):
    wrong_english=[]
    doc = input_string
    doc_nlp = nlp(doc)
    for sentences in doc_nlp.sents:
      wrong_english.append(str(sentences))
    tokenized=tokenizer(
        wrong_english,
        padding="longest",
        max_length=MAX_LENGTH,
        truncation=True,
        return_tensors='pt'
    )
    out=pretrained_model.generate(**tokenized, max_length=128)
    right_english=[]
    for i in range(len(wrong_english)):
      right_english.append(tokenizer.decode(out[i], skip_special_tokens=True))
    doc2=''.join(right_english)
    summary = summarizer(str(doc2), max_length=130, min_length=30, do_sample=False)
    return doc2, summary

# Streamlit application
st.title('Grammar correction')
user_input = st.text_input('Enter a sentence:')

if user_input:
    corrected , summarized = grammarly(user_input)
    st.write('Corrected grammar', corrected)
    st.write('Improved vocabulary', summarized)

# Run the Streamlit app
!streamlit run app.py &>/dev/null&

# Use ngrok to create a public URL
from pyngrok import ngrok
public_url = ngrok.connect(port='8501')
print(f'Access your Streamlit app at: {public_url}')


In [ ]:
%%writefile app.py

Overwriting app.py
